# Imports and Data

## Import Libraries

In [3]:
import os
import numpy as np
import tensorflow as tf
import random

from numpy.random import randint

import tifffile as tiff
from osgeo import gdal

import cv2 as cv
from IPython.display import Image

from keras.models import Model
from keras.layers import Input, Activation, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.optimizers import Adam

from skimage import transform, filters

## Reshape Pictures
No need to execute, if pictures are already reshaped to (224, 224, 3)


In [24]:
def resize_images():
    trajectory_folders = ['cropped_resized']
    trajectory_numbers = ['20', '22', '24']
    subfolders = ['cut_1', 'cut_2', 'cut_3', 'cut_4']
    for trajectory in trajectory_folders:
        for number in trajectory_numbers:
            for subfolder in subfolders:
                i = 1
                while i <= 50:
                    path_read = "/content/drive/MyDrive/MLDM/" + trajectory + "/" + trajectory + "/" + number + "/" + subfolder + "/" + str(i) + ".jpg"
                    print(path_read)
                    img = cv.imread(path_read)
                    resized_image = cv.resize(img, (224, 224))
                    path_write = "/content/drive/MyDrive/MLDM/" + trajectory + "/" + trajectory + "/" + number + "/" + subfolder + "/" + str(i) + ".jpg"
                    cv.imwrite(path_write, resized_image)
                    i += 1

In [25]:
resize_images()

/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/1.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/2.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/3.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/4.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/5.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/6.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/7.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/8.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/9.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/10.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/11.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/12.jpg
/content/drive/MyDrive/MLDM/cropped_resized/cropped_resized/20/cut_1/13.jpg
/content/drive/MyDriv

In [30]:
def crop_images():
    trajectory_folders = ['cropped_cropped']
    trajectory_numbers = ['20', '22', '24']
    subfolders = ['cut_1', 'cut_2', 'cut_3', 'cut_4']
    for trajectory in trajectory_folders:
        for number in trajectory_numbers:
            for subfolder in subfolders:
                i = 1
                while i <= 50:
                    path_read = "/content/drive/MyDrive/MLDM/" + trajectory + "/" + trajectory + "/" + number + "/" + subfolder + "/" + str(i) + ".jpg"
                    print(path_read)
                    img = cv.imread(path_read)
                    cropped_img = img[0:224, 0:224]
                    path_write = "/content/drive/MyDrive/MLDM/" + trajectory + "/" + trajectory + "/" + number + "/" + subfolder + "/" + str(i) + ".jpg"
                    cv.imwrite(path_write, cropped_img)
                    i += 1

In [31]:
crop_images()

/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/1.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/2.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/3.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/4.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/5.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/6.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/7.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/8.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/9.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/10.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/11.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/12.jpg
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/13.jpg
/content/drive/MyDriv

# Change Detection in VHR Imagery with severe co-registration errors using Deep Learning

**Code of the paper**: https://github.com/vkristoll/change-detection-autoencoder/blob/main/coregister.py


## Review

### Define Autoencoder

In [82]:
#Create function to define the autoencoder model
def autoencoder():
    #Define the patch size
    original_img_size = (224, 224, 4)

    #encoder
    input_img = Input(shape=original_img_size)
    x = Conv2D(64, (3, 3), padding='same')(input_img)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), padding='same')(x)
    encoded = Activation('relu')(x)

    #decoder
    x = Conv2DTranspose(32, (4,4), strides=(2,2), padding='same')(encoded)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same')(x)
    x = Activation('relu')(x)
    x= Conv2D(4, (3, 3), padding='same')(x)
    decoded = Activation('sigmoid')(x)

    #Define the model
    model = Model(input_img, decoded)

    #Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

#Define the autoencoder model
model=autoencoder()
print(model.summary())

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 4)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 224, 224, 64)      2368      
                                                                 
 activation_12 (Activation)  (None, 224, 224, 64)      0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 112, 112, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 112, 112, 32)      18464     
                                                                 
 activation_13 (Activation)  (None, 112, 112, 32)      0         
                                                          

### Create change maps

In [83]:
def generate_changemap(x, trajectory_folder, trajectory_number, subfolder):

    #Define paths to 2 folders that contain the patches of the 1st and 2nd date (224x224x4 px)
    #Create sorted list of the files
    dir1_path="/content/drive/MyDrive/MLDM/"+ trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/" + subfolder + "/"
    dir1_list=os.listdir(dir1_path)
    dir1_sorted_list=sorted(dir1_list)

    dir2_path="/content/drive/MyDrive/MLDM/"+ trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/" + subfolder + "/"
    dir2_list=os.listdir(dir2_path)
    dir2_sorted_list=sorted(dir2_list)

    #Define path to folder to store the change maps
    dir3_path="/content/drive/MyDrive/MLDM/"+ trajectory_folder + "/result_" + trajectory_number + "_" + subfolder + "/"

    #Count number of files
    number_files=len(dir1_list)

    for i in range(0, number_files-1, 1):
        #Access the first date folder
        os.chdir(dir1_path)
        #Read the first date image
        im1 = gdal.Open(dir1_sorted_list[i])
        im1=np.array(im1.ReadAsArray())
        #Change the order of the axes to width,height,channels
        im1=im1.transpose(1,2,0)
        im1 = np.concatenate((im1, np.zeros_like(im1[..., :1])), axis=-1)
        #Convert values to [0,1]
        im1=im1/255
        #Expand the dimensions so that the model can read it (batch_size=1)
        im1=np.expand_dims(im1,0)

        #Store the file name and define name of the change map
        if x == True:
            filename=os.path.basename("changemap_big" + str(i) + "_" + str(i+1) + ".tiff")
        else:
            filename=os.path.basename("changemap" + str(i) + "_" + str(i+1) + ".tiff")
        print(filename)

        os.chdir(dir2_path)
        im2 = gdal.Open(dir2_sorted_list[i+1])
        im2=np.array(im2.ReadAsArray())
        im2=im2.transpose(1,2,0)
        im2 = np.concatenate((im2, np.zeros_like(im2[..., :1])), axis=-1)
        im2= im2/255
        im2=np.expand_dims(im2,0)

        F1=feat(im1) #Features from image patch 1
        #Calculate the square value of the feature maps
        F1=tf.square(F1)
        F2=feat(im2) #Features from image patch 2
        F2=tf.square(F2)
        #Subtract the feature maps from the 2 dates
        d=tf.subtract(F1,F2)
        d=tf.square(d)
        #Create the change map by summing values from the feature maps
        d=tf.reduce_sum(d,axis=3)

        #Resize the change map
        dis=d.numpy()
        dis=dis.transpose(1,2,0)
        dis = tf.image.resize(dis,[224,224], method="nearest")
        dis=dis.numpy()
        dis=np.resize(dis,[224,224])

        #Save the change map
        os.chdir(dir3_path)
        tiff.imsave(filename,dis)

### Change format to .png

In [84]:
def convert_to_png(trajectory_folder, trajectory_number, subfolder):
    #Define the folder path that contains the change maps with the float values
    #Create list of the files
    dir1_path="/content/drive/MyDrive/MLDM/" + trajectory_folder + "/result_" + trajectory_number + "_" + subfolder +"/"
    dir1_list=os.listdir(dir1_path)
    #Define folder path to store the binary change maps
    dir2_path="/content/drive/MyDrive/MLDM/" + trajectory_folder + "/result_" + trajectory_number + "_" + subfolder + "_png/"
    #Count the number of the files
    number_files=len(dir1_list)

    #Iterate over the files and create the binary change maps
    for i in range (number_files):

        #Print the index of the file
        print(" The file index is %s" %i)

        #Access the directory of the change maps (float)
        os.chdir(dir1_path)
        #Read image
        im1 = gdal.Open(dir1_list[i])
        im1=np.array(im1.ReadAsArray())

        #Store the file name and define the name of the binary change map
        filename=os.path.basename(dir1_list[i])
        filename2=filename.split('.')[0] + ".png"

        #Access the directory of the binary change maps
        os.chdir(dir2_path)

        cv.imwrite(filename2, im1)

In [85]:
trajectory_folders = ['cropped_cropped', 'cropped_resized']
trajectory_numbers = ['20', '22', '24']
subfolders = ['cut_1', 'cut_2', 'cut_3', 'cut_4']

for folder_name in trajectory_folders:
    for number in trajectory_numbers:
        #Load the weights
        #The autoencoder model is  defined in "autoencoder_training.py"
        model.load_weights("/content/drive/MyDrive/MLDM/" + folder_name + "/" + folder_name + "/" + number + "/new_weights.h5")

        #Run the line below to find the indexes of the layers
        layer_names = [layer.name for layer in model.layers]

        #Define submodels that calculate the model predictions for the selected indexes
        model1 = Model(inputs=model.inputs, outputs=model.layers[3].output)
        model2 = Model(inputs=model.inputs, outputs=model.layers[6].output)
        model3 = Model(inputs=model.inputs, outputs=model.layers[10].output)
        model4 = Model(inputs=model.inputs, outputs=model.layers[12].output)

        #Define function that resizes the multilevel feature maps and concatenates them
        def feat(x_train):

            feat1 = model1.predict(x_train)
            feat2 = model2.predict(x_train)
            feat3 = model3.predict(x_train)
            feat4 = model4.predict(x_train)

            x1 = tf.image.resize(feat1,[128,128])
            x2 = tf.image.resize(feat2,[128,128])
            x3 = tf.image.resize(feat3,[128,128])
            x4 = tf.image.resize(feat4,[128,128])

            F = tf.concat([x2,x1,x4,x3],3)
            return F

        for subfolder in subfolders:
            print("**********************************************************")
            print("/content/drive/MyDrive/MLDM/"+ folder_name + "/"+ folder_name + "/" + number + "/" + subfolder + "/")
            print("**********************************************************")
            generate_changemap(False, folder_name, number, subfolder)
            convert_to_png(folder_name, number, subfolder)

**********************************************************
/content/drive/MyDrive/MLDM/cropped_cropped/cropped_cropped/20/cut_1/
**********************************************************
changemap0_1.tiff
1/1 [==============================] - 0s 72ms/step


1/1 [==============================] - 0s 18ms/step


<ipython-input-83-92f2b6279112>:68: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(filename,dis)


changemap1_2.tiff
1/1 [==============================] - 0s 19ms/step
changemap2_3.tiff
1/1 [==============================] - 0s 17ms/step
changemap3_4.tiff
1/1 [==============================] - 0s 30ms/step
changemap4_5.tiff
1/1 [==============================] - 0s 29ms/step
changemap5_6.tiff
1/1 [==============================] - 0s 27ms/step
changemap6_7.tiff
1/1 [==============================] - 0s 24ms/step
changemap7_8.tiff
1/1 [==============================] - 0s 39ms/step
changemap8_9.tiff
1/1 [==============================] - 0s 20ms/step
changemap9_10.tiff
1/1 [==============================] - 0s 17ms/step
changemap10_11.tiff
1/1 [==============================] - 0s 19ms/step
changemap11_12.tiff
1/1 [==============================] - 0s 21ms/step
changemap12_13.tiff
1/1 [==============================] - 0s 20ms/step
changemap13_14.tiff
1/1 [==============================] - 0s 17ms/step
changemap14_15.tiff
1/1 [==============================] - 0s 19ms/step
changemap

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
1/1 [==============================] - 0s 24ms/step
changemap47_48.tiff
1/1 [==============================] - 0s 31ms/step
changemap48_49.tiff
1/1 [==============================] - 0s 26ms/step
 The file index is 0
 The file index is 1
 The file index is 2
 The file index is 3
 The file index is 4
 The file index is 5
 The file index is 6
 The file index is 7
 The file index is 8
 The file index is 9
 The file index is 10
 The file index is 11
 The file index is 12
 The file index is 13
 The file index is 14
 The file index is 15
 The file index is 16
 The file index is 17
 The file index is 18
 The file index is 19
 The file index is 20
 The file index is 21
 The file index is 22
 The file index is 23
 The file index is 24
 The file index is 25
 The file index is 26
 The file index is 27
 The file index is 28
 The file index is 29
 The file index is 30
 The file index is 31
 The file index is 32
 The file 

### Train Autoencoder

In [76]:
trajectory_folder = 'cropped_resized'   # one of the following: ['cropped_cropped', 'cropped_resized']
trajectory_number = '24'                # one of the following: ['20', '22', '24']

In [77]:
#Creating training data

#Import libraries
import os
import numpy as np
from osgeo import gdal

'''Define path to 4 folders that contain first date patches (224x224x3 px).'''

#Create list with the names of the patches
dir1_path="/content/drive/MyDrive/MLDM/" + trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/cut_1/"
dir1_list=os.listdir(dir1_path)

dir1_path="/content/drive/MyDrive/MLDM/" + trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/cut_2/"
dir2_list=os.listdir(dir2_path)

dir1_path="/content/drive/MyDrive/MLDM/" + trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/cut_3/"
dir3_list=os.listdir(dir3_path)

dir1_path="/content/drive/MyDrive/MLDM/" + trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/cut_4/"
dir4_list=os.listdir(dir4_path)

#Create list that contains the paths of the 4 folders
dir_path_list=[dir1_path,dir2_path,dir3_path, dir4_path]

#Create list that contains the 4 lists of the names of the patches
dir_sorted_list=[sorted(dir1_list),sorted(dir2_list),sorted(dir3_list),sorted(dir4_list)]
print(dir_sorted_list)
#Create list that stores the number of files (patches) in each folder
number_files_list=[len(dir1_list),len(dir2_list),len(dir3_list),len(dir4_list)]

#Create list that contains float arrays to store the patches for each study area

train_list=[np.float16(np.zeros((len(dir1_list),224,224,4))),np.float16(np.zeros((len(dir2_list),224,224,4))),
            np.float16(np.zeros((len(dir3_list),224,224,4))),np.float16(np.zeros((len(dir4_list),224,224,4)))]

[['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '3.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '4.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '5.jpg', '50.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg'], ['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '3.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '4.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '5.jpg', '50.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg'], ['1.jpg', '10

In [78]:
#Add the patches to the train_list
for i in range (3):
    os.chdir(dir_path_list[i])
    for k in range(0, number_files_list[i],1):
        #Print the index of the patch
        print(f"x_train {i}: k is {k}")
        #Read the patch
        x_1 = gdal.Open(dir_sorted_list[i][k])
        #Convert it to an array
        x_1 = np.array(x_1.ReadAsArray())
        #Change the order of the axes to width,height,channels
        x_1 = x_1.transpose(1,2,0)
        #Reshape
        x_1 = np.concatenate((x_1, np.zeros_like(x_1[..., :1])), axis=-1)
        #Convert values to [0,1]
        x_1 = x_1/255
        #Add the patch to the train_list
        train_list[i][k,:,:,:]=x_1

x_train 0: k is 0
x_train 0: k is 1
x_train 0: k is 2
x_train 0: k is 3
x_train 0: k is 4
x_train 0: k is 5
x_train 0: k is 6
x_train 0: k is 7
x_train 0: k is 8
x_train 0: k is 9
x_train 0: k is 10
x_train 0: k is 11
x_train 0: k is 12
x_train 0: k is 13
x_train 0: k is 14
x_train 0: k is 15
x_train 0: k is 16
x_train 0: k is 17
x_train 0: k is 18
x_train 0: k is 19
x_train 0: k is 20
x_train 0: k is 21
x_train 0: k is 22
x_train 0: k is 23
x_train 0: k is 24
x_train 0: k is 25
x_train 0: k is 26
x_train 0: k is 27
x_train 0: k is 28
x_train 0: k is 29
x_train 0: k is 30
x_train 0: k is 31
x_train 0: k is 32
x_train 0: k is 33
x_train 0: k is 34
x_train 0: k is 35
x_train 0: k is 36
x_train 0: k is 37
x_train 0: k is 38
x_train 0: k is 39
x_train 0: k is 40
x_train 0: k is 41
x_train 0: k is 42
x_train 0: k is 43
x_train 0: k is 44
x_train 0: k is 45
x_train 0: k is 46
x_train 0: k is 47
x_train 0: k is 48
x_train 0: k is 49
x_train 1: k is 0
x_train 1: k is 1
x_train 1: k is 2
x_trai

In [79]:
def batch_generator():

    #Define batch size
    batch_size=8
    #Select randomly one of the 3 study areas
    randomindexim=randint(3)

    #Select random sample of indexes equal to the batch size from the randomly selected study area
    if randomindexim==0:
       randomindexsample=random.sample(range(0,len(dir1_list)),batch_size)

    elif  randomindexim==1:
       randomindexsample=random.sample(range(0,len(dir2_list)),batch_size)

    elif  randomindexim==2:
       randomindexsample=random.sample(range(0,len(dir3_list)),batch_size)

    #Create array with patches equal to the batch size
    X=np.float16(np.zeros((batch_size,224,224,4)))
    c=0
    for i in range(1, batch_size, 1):
        c=c+1
        X[c-1,:,:,:]=train_list[randomindexim][randomindexsample[i],:,:,:]

    # Return training batch. It's the same for the input and the output because the model is an autoencoder.
    return X,X

In [80]:
#Define the number of epochs
epochs=400

#Define the number of training steps
#dir1_list, dir2_list, dir3_list, dir4_list are defined in "create_training_data.py"
train_steps=int((len(dir1_list)+ len(dir2_list) + len(dir3_list))/14)

for i in range (epochs):
    for j in range (train_steps):
        #batch_generator is defined in "training_batch_generator.py"
        X,y = batch_generator()
        loss=model.train_on_batch(X,y)

        print('>%d, %d/%d, d=%.3f' % (i+1, j+1, train_steps, loss))

>1, 1/10, d=0.000
>1, 2/10, d=0.000
>1, 3/10, d=0.000
>1, 4/10, d=0.000
>1, 5/10, d=0.000
>1, 6/10, d=0.000
>1, 7/10, d=0.000
>1, 8/10, d=0.001
>1, 9/10, d=0.001
>1, 10/10, d=0.001
>2, 1/10, d=0.000
>2, 2/10, d=0.000
>2, 3/10, d=0.000
>2, 4/10, d=0.001
>2, 5/10, d=0.000
>2, 6/10, d=0.001
>2, 7/10, d=0.001
>2, 8/10, d=0.001
>2, 9/10, d=0.001
>2, 10/10, d=0.000
>3, 1/10, d=0.000
>3, 2/10, d=0.000
>3, 3/10, d=0.001
>3, 4/10, d=0.000
>3, 5/10, d=0.000
>3, 6/10, d=0.001
>3, 7/10, d=0.000
>3, 8/10, d=0.000
>3, 9/10, d=0.000
>3, 10/10, d=0.001
>4, 1/10, d=0.001
>4, 2/10, d=0.000
>4, 3/10, d=0.000
>4, 4/10, d=0.001
>4, 5/10, d=0.000
>4, 6/10, d=0.000
>4, 7/10, d=0.000
>4, 8/10, d=0.000
>4, 9/10, d=0.000
>4, 10/10, d=0.001
>5, 1/10, d=0.000
>5, 2/10, d=0.001
>5, 3/10, d=0.001
>5, 4/10, d=0.000
>5, 5/10, d=0.000
>5, 6/10, d=0.000
>5, 7/10, d=0.000
>5, 8/10, d=0.001
>5, 9/10, d=0.000
>5, 10/10, d=0.001
>6, 1/10, d=0.000
>6, 2/10, d=0.000
>6, 3/10, d=0.000
>6, 4/10, d=0.000
>6, 5/10, d=0.000
>6, 6

In [81]:
#Safe model weights as .h5
model.save_weights("/content/drive/MyDrive/MLDM/" + trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/new_weights.h5")

#Safe model weights as .txt
model.load_weights("/content/drive/MyDrive/MLDM/" + trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/new_weights.h5")

np.set_printoptions(threshold=np.inf)
with open("/content/drive/MyDrive/MLDM/" + trajectory_folder + "/" + trajectory_folder + "/" + trajectory_number + "/new_weights.txt", 'w') as f:
    for layer in model.layers:
       f.write(str(layer.get_config()))
       f.write(str(layer.get_weights()))
print("Weights saved!")

Weights saved!


## Watch a Changemap

In [91]:
path_read = "/content/drive/MyDrive/MLDM/cropped_resized/result_22_cut_1_png/changemap21_22.png"
print(path_read)
img = cv.imread(path_read)
print(img)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  [ 51  51  51]
  [ 10  10  10]
  [ 10  10  10]
  [  5   5   5]
  [  6   6   6]
  [  6   6   6]
  [ 11  11  11]
  [ 11  11  11]
  [ 21  21  21]
  [ 21  21  21]
  [ 37  37  37]
  [ 29  29  29]
  [ 29  29  29]
  [  5   5   5]
  [  5   5   5]
  [  3   3   3]
  [  3   3   3]
  [  3   3   3]
  [ 10  10  10]
  [ 10  10  10]
  [ 17  17  17]
  [ 17  17  17]
  [ 18  18  18]
  [ 18  18  18]
  [ 16  16  16]
  [ 18  18  18]
  [ 18  18  18]
  [ 14  14  14]
  [ 14  14  14]
  [ 10  10  10]
  [ 10  10  10]
  [ 13  13  13]
  [ 15  15  15]
  [ 15  15  15]
  [ 19  19  19]
  [ 19  19  19]
  [ 15  15  15]
  [ 15  15  15]
  [ 13  13  13]
  [ 15  15  15]
  [ 15  15  15]
  [ 11  11  11]
  [ 11  11  11]
  [ 14  14  14]
  [ 14  14  14]
  [ 13  13  13]
  [ 23  23  23]
  [ 23  23  23]
  [ 22  22  22]
  [ 22  22  22]]

 [[ 11  11  11]
  [ 11  11  11]
  [  8   8   8]
  [  8   8   8]
  [  8   8   8]
  [ 10  10  10]
  [ 10  10  10]
  [ 10  